# Pré processamento dos conjuntos de dados

In [5]:
import pandas as pd
import os
from utils import get_workdir, SEED
from binary_encoders import Thermometer, OneHot
from sklearn.model_selection import train_test_split

data = pd.read_csv(f"../dataset/hepatitis/hcvdat0.csv").drop("Unnamed: 0", axis="columns").dropna()
data

,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
0,0=Blood Donor,32,m,38.5,52.5,7.7,22.1,7.5,6.93,3.23,106.0,12.1,69.0
1,0=Blood Donor,32,m,38.5,70.3,18.0,24.7,3.9,11.17,4.80,74.0,15.6,76.5
2,0=Blood Donor,32,m,46.9,74.7,36.2,52.6,6.1,8.84,5.20,86.0,33.2,79.3
3,0=Blood Donor,32,m,43.2,52.0,30.6,22.6,18.9,7.33,4.74,80.0,33.8,75.7
4,0=Blood Donor,32,m,39.2,74.1,32.6,24.8,9.6,9.15,4.32,76.0,29.9,68.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
608,3=Cirrhosis,58,f,34.0,46.4,15.0,150.0,8.0,6.26,3.98,56.0,49.7,80.6
609,3=Cirrhosis,59,f,39.0,51.3,19.6,285.8,40.0,5.77,4.51,136.1,101.1,70.5
610,3=Cirrhosis,62,f,32.0,416.6,5.9,110.3,50.0,5.57,6.30,55.7,650.9,68.5
611,3=Cirrhosis,64,f,24.0,102.8,2.9,44.4,20.0,1.54,3.02,63.0,35.9,71.3


In [4]:
data_desc = data.describe()
data_desc

,Age,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
count,615.000000,614.000000,597.000000,614.000000,615.000000,615.000000,615.000000,605.000000,615.000000,615.000000,614.000000
mean,47.408130,41.620195,68.283920,28.450814,34.786341,11.396748,8.196634,5.368099,81.287805,39.533171,72.044137
std,10.055105,5.780629,26.028315,25.469689,33.090690,19.673150,2.205657,1.132728,49.756166,54.661071,5.402636
min,19.000000,14.900000,11.300000,0.900000,10.600000,0.800000,1.420000,1.430000,8.000000,4.500000,44.800000
25%,39.000000,38.800000,52.500000,16.400000,21.600000,5.300000,6.935000,4.610000,67.000000,15.700000,69.300000
50%,47.000000,41.950000,66.200000,23.000000,25.900000,7.300000,8.260000,5.300000,77.000000,23.300000,72.200000
75%,54.000000,45.200000,80.100000,33.075000,32.900000,11.200000,9.590000,6.060000,88.000000,40.200000,75.400000
max,77.000000,82.200000,416.600000,325.300000,324.000000,254.000000,16.410000,9.670000,1079.100000,650.900000,90.000000


In [39]:
def create_column_encoder(data:pd.DataFrame)->dict:
    column_encoder = {}
    data_desc = data.describe()

    for column in data.columns:
        if column in data_desc.columns: # this is a numerical column
            column_encoder[column] = Thermometer(min=data_desc[column]["min"],max=data_desc[column]["max"],num_bits=32)
        else: # this is a categorical column
            column_encoder[column] = OneHot(list(data[column].unique()))
    
    return column_encoder

In [41]:
tgt = "Attack_type"

train_ft = data.drop(columns=["Attack_type"])
train_labels = data["Attack_type"]

column_encoder = create_column_encoder(train_ft)
column_encoder

{'id.orig_p': <binary_encoders.Thermometer at 0x7ec7cee00f50>,
 'id.resp_p': <binary_encoders.Thermometer at 0x7ec7cee03710>,
 'proto': <binary_encoders.OneHot at 0x7ec7cee01610>,
 'service': <binary_encoders.OneHot at 0x7ec7cee009e0>,
 'flow_duration': <binary_encoders.Thermometer at 0x7ec7cee00710>,
 'fwd_pkts_tot': <binary_encoders.Thermometer at 0x7ec7cee00680>,
 'bwd_pkts_tot': <binary_encoders.Thermometer at 0x7ec7cee02420>,
 'fwd_data_pkts_tot': <binary_encoders.Thermometer at 0x7ec7cee00380>,
 'bwd_data_pkts_tot': <binary_encoders.Thermometer at 0x7ec7cee00890>,
 'fwd_pkts_per_sec': <binary_encoders.Thermometer at 0x7ec7cee00c80>,
 'bwd_pkts_per_sec': <binary_encoders.Thermometer at 0x7ec7cee02300>,
 'flow_pkts_per_sec': <binary_encoders.Thermometer at 0x7ec7cee022a0>,
 'down_up_ratio': <binary_encoders.Thermometer at 0x7ec7cee01a00>,
 'fwd_header_size_tot': <binary_encoders.Thermometer at 0x7ec7cee00230>,
 'fwd_header_size_min': <binary_encoders.Thermometer at 0x7ec7cee01280>,

In [42]:
for column in column_encoder:
    train_ft[column] = [column_encoder[column].bitarray_encode(elem) for elem in train_ft[column]]
train_ft

/home/arthur/Documents/Trabs/GreenAI/wisard_grai/src/binary_encoders.py:18: RuntimeWarning: invalid value encountered in scalar divide
  bits_activated = int(math.ceil(((data - self.min)/ self.interval ) * self.num_bits))


ValueError: cannot convert float NaN to integer

In [ ]:
from bitarray import bitarray
def binarize_dataset(data, column_encoder):
    for column in column_encoder:
        train_ft[column] = [column_encoder[column].bitarray_encode(elem) for elem in train_ft[column]]
    train_mtx = []
    for idx, row in train_ft.iterrows():
        line = bitarray()
        for item in row:
            line.extend(item)
        train_mtx.append(line)
    return train_mtx

[bitarray('10011111111111111111000000000000000111111111111111110000000000000001110000000000000000000000000000011111100000000000000000000000000111110000000000000000000000000001111100000000000000000000000000011111000000000000000000000000000'), bitarray('10011111111111100000000000000000000111111111111000000000000000000001110000000000000000000000000000011100000000000000000000000000000111000000000000000000000000000001110000000000000000000000000000011100000000000000000000000000000'), bitarray('01011111111111111111111000000000000111111111111111111110000000000001111000000000000000000000000000011111111000000000000000000000000111111000000000000000000000000001111110000000000000000000000000011111110000000000000000000000000'), bitarray('10011111111111111110000000000000000111111111111111110000000000000001111000000000000000000000000000011111100000000000000000000000000111110000000000000000000000000001111100000000000000000000000000011111000000000000000000000000000'), bitarray('0011111111111110000000000